In [1]:
import numpy as np
import plotly.graph_objects as go

import sys
sys.path.append('../')
from data_generation import sigmoid_fn, dec_exp_fn, linear_fn, exp_fn, log_fn
from roelfes_emulator import RoelfesEmulator
import pandas as pd
from preprocessing import preprocessor
from botorch.test_functions.synthetic import SyntheticTestFunction
import torch

import matplotlib.pyplot as plt
import numpy as np
import plotly.subplots as sp
import plotly.graph_objects as go
from bayesian_opt import BayesianOptimization

In [2]:
# Parameters
mu = [0.25, 0.5, 0.75]
tau = 0.1

# Function parameters: (a, b) for sigmoid, linear, exp, log
params = {
    'linear': [(0.25, 0.1), (0.4, 2)],
    'log': [(0.9, 0.1)]
}

In [3]:
observations = 300
n_features = 2

In [4]:

# Import your RoelfesEmulator class here
# from your_module import RoelfesEmulator

# Setup
observations = 100
feature1 = np.linspace(0, 1, observations)
feature2_vals = [0, 0.5, 1]  # Pick 3 values

# Initialize emulator
em = RoelfesEmulator(dim=2, space_complexity=10, use_torch=False)
t_vals = em.t_vals  # use internal t_vals

# Create subplots: 1 row, 3 cols
fig = sp.make_subplots(rows=1, cols=3, subplot_titles=[f"x2 = {val}" for val in feature2_vals])

for i, x2_val in enumerate(feature2_vals):
    # Create features
    fixed_feature2 = np.full_like(feature1, x2_val)
    features = np.column_stack((feature1, fixed_feature2))  # shape (100, 2)

    # Evaluate using emulator
    Y = em.yield_calc(features)  # shape (100, 300)

    # Add heatmap to subplot (transpose for proper axis orientation)
    fig.add_trace(
        go.Heatmap(z=Y.T, x=feature1, y=t_vals, colorscale='Viridis', showscale=(i == 2)),
        row=1, col=i + 1
    )

# Layout tweaks
fig.update_layout(
    title="f(x, t) Heatmaps for Different x2 (feature2) Values",
    height=500,
    width=1200,
)

fig.show()


In [ ]:
def bayes_opt_roelfes_coef(objs, q, q_sampling_method="Monte Carlo"):
    lin_df = objs.drop(['y3','y4','y5','y6'],axis=1)
    dec_exp_df = objs.drop(['y1','y2'], axis=1)

    q1 = q // 2
    q2 = q - q1

    lin_bo_model = BayesianOptimization().fit(lin_df, y=['y1', 'y2'])
    c1 = lin_bo_model.candidates(q1, export_df=True, q_sampling_method=q_sampling_method)
    dec_exp_bo_model = BayesianOptimization().fit(dec_exp_df, y=['y3','y4','y5','y6'], 
                                                optim_direc=['max','max', 'min','max'])
    c2 = dec_exp_bo_model.candidates(q2, export_df=True,q_sampling_method=q_sampling_method)
    cand = pd.concat((c1, c2)).fillna(0)
    return cand

In [6]:
x = torch.tensor(np.random.rand(40,3))
em = RoelfesEmulator(dim=3, objective="coef", space_complexity=10, use_torch=True)
objs = em(x)

# Create column names
x_columns = [f'x{i+1}' for i in range(x.shape[1])]
y_columns = [f'y{i+1}' for i in range(objs.shape[1])]

# Combine into one DataFrame
df = pd.DataFrame(
    data = torch.cat((x, objs), dim=1).numpy(),
    columns = x_columns + y_columns
)

bayes_opt_roelfes_coef(df, 3)


c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\acquisition\multi_objective\analytic.py:90: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\acquisition\multi_objective\monte_carlo.py:111: NumericsWarning:

qExpectedHypervolumeImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 qExpectedHypervolumeImprovement 	 --> 	 qLogExpectedHypervolumeImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.



,x1,x2,x3,y1,y2,y3,y4,y5,y6
0,0.00,0.13,0.03,0.000142,0.653731,0.000000,0.000000,0.000000,0.000000
0,0.71,0.44,0.80,0.000000,0.000000,0.392433,0.005194,-404.509892,0.165914
1,0.65,0.90,0.53,0.000000,0.000000,0.796981,0.006656,-2590.228032,0.176221


In [20]:
x = torch.tensor(np.random.rand(40,3))
em = RoelfesEmulator(dim=3, objective="normalized yield gradient", space_complexity=10, use_torch=True)
objs  = em(x)
objs
# Create column names
x_columns = [f'x{i+1}' for i in range(x.shape[1])]
y_columns = [f'y{i+1}' for i in range(objs.shape[1])]

# Combine into one DataFrame
df = pd.DataFrame(
    data = torch.cat((x, objs), dim=1).numpy(),
    columns = x_columns + y_columns
)
model = BayesianOptimization().fit(df, ['y1'])
cand, pred = model.candidates(q=3)
cand

array([[0.57, 0.22, 0.17],
       [0.73, 0.71, 0.32],
       [1.  , 0.79, 0.23]])